# Bus Model Information
This notebook is intended to act as a location where values and sources for different bus, trip, and ESS models are stored. 

In [1]:
# General Purpose:
import sys
sys.path.append("../src/reRoute_Dynamics_Core/")
import Object_Params as op
bus_directory = "./KC_Example_Data/Saved_Objects/Busses/"
trip_directory =  "./KC_Example_Data/Saved_Objects/Trips/"
ESS_directory =  "./KC_Example_Data/Saved_Objects/ESSes/"

a_prof_path = "KC_Example_Data/Acceleration_Profiles/Braunschweig_Acceleration.csv"#Interpolated from @NREL_Drive_Cylces

In [2]:
base_trip = op.Trip(pass_mass = 84, #kg, calculated from @CDC_Human_Mass
                    limit_MOE = .15, # percent of the speed limit the value can be within. 
                    signal_rest = 65/2, # seconds, based on @WSDOT_Signals
                    signal_chance = .541666, # fraction o fthe time the bus will hit a red light when passing a signal. This is calculated based on @WSDOT_Signals
                    stop_rest = 7, # seconds per passenger boarding. This is a Guess. 
                    sign_rest = 7, # seconds. This is a guess of how long a bus will stop at a stop sign.
                    end_rest = 10, # seconds.amount of time the bus rests at the beginning or end of a route before bus is turned off.
                    air_density = 1.22, # based on seattle typical temperature of 12.7 degrees C. @Air_Density
                    wind_speed = 1.78, #m/s, based on typical wind speed in seattle @Weatherspark_Seattle
                    wind_heading = 'SE', #per @Weatherspark_Seattle
                    temperature = 12.7, #deg C, per @Weatherspark_Seattle
                    interp_length = 10, # meters, guess. 
                    mean_ridership = 3.5, # People. Back of napkin from @KC_Rider_Stats and other KCM route data
                    seed = 42, # random seed, subject to change
                    lg=43, #savgol param
                    deg = 3, #savgol param
                    stop_margin = 1 #m/s, margin for the velocity to be considered 'stopped'
                   )
base_trip.save(trip_directory+"base_seattle_trip.txt")

'./KC_Example_Data/Saved_Objects/Trips/base_seattle_trip.txt'

In [3]:
# XDE35 BEV version: 
XDE35 = op.Bus(bus_mass = 13920, #kg, @NF_Xcelsior
               frontal_width = 2.6, #m,@NF_Xcelsior
               frontal_height = 3.38, #m, @NF_Xcelsior
               drag_coeff = .79, # Unitless. Many suspect papers cite .6, but best bet was one citing ORNL bus database saying .79, @Gao_Et_Al
               friction_coeff = .01, # unitless, otherwise known as rolling resistance. @Rolling_Resistance
               braking_accel = 1.5, #1.5m/s^2, handbrake required to stop from 20 mph, over no set distance. emergency brake must be capable of up to 6.5m/s^2 (20 mph over 20 ft).@APTA_Braking_Standards
               br_factor = .5, # driver braking aggression. Variable based on driver.
               i_factor = 1.1, # unitless, accounts for wheels, driveshaft, etc, cited from @Gallet
                               # however, there is no substantive explanation as to how it was calculated/obtained.
               max_dist = 304.8, # m, expected stopping distance for a bust from 60 mph calculated from google map offramp length measurements of I-5.
               a_prof_path =a_prof_path, # based on the Braunschweig drive cycle @NREL_Drive_Cycles
               max_acc = .4, # m/s^2, chosen due to it being a decent extension of existing drive cycle. subject to change.
               max_dt = .5, #s, timestep for that extension. 
               max_P = 300000 # W, @Siemens_Traction_Motor 
              )

# Saving the object.
XDE35.save(bus_directory + "XDE35.txt")


# The XDE35 uses the @Siemens_Traction_Motor
# Cooling system: @E-Fan_Cooling
# Approx 10080 watts at max cooling
# HVAC system, @Thermoking_TE18
# Approx 3k watts at max hvac
# thus, a reasonable nominal load is 8k? bottom is 4k?
XDE35_ESS = op.ESS(motor_eff = .91, # @Gallet. No clear source on where these drivetrain, inverter, and motor efficiencies came from. Can't find TB200 specs yet.
                   inverter_eff = .97, # @Gallet
                   aux_eff = .89, # Assumption.
                   regen_eff = .6, # Assumption.
                   simple_load = 8000,
                   max_regen = -200000 #w, finally, some good sourcable values.@XDE35_Manual
                   # W @Siemens_Traction_Motor
                  )

XDE35_ESS.save(ESS_directory + "XDE35_ESS.txt")

'./KC_Example_Data/Saved_Objects/ESSes/XDE35_ESS.txt'

In [4]:
# XDE60 BEV version: 
XDE60 = op.Bus(bus_mass = 20638, #kg, @NF_Xcelsior
               frontal_width = 2.6, #m,@NF_Xcelsior
               frontal_height = 3.38, #m, @NF_Xcelsior
               drag_coeff = .79, # Unitless. Many suspect papers cite .6, but best bet was one citing ORNL bus database saying .79, @Gao_Et_Al
               friction_coeff = .01, # unitless, otherwise known as rolling resistance. @Rolling_Resistance
               braking_accel = 1.5, #1.5m/s^2, handbrake required to stop from 20 mph, over no set distance. emergency brake must be capable of up to 6.5m/s^2 (20 mph over 20 ft).@APTA_Braking_Standards
               br_factor = .5, # driver braking aggression. Variable based on driver.
               i_factor = 1.1, # unitless, accounts for wheels, driveshaft, etc, cited from @Gallet
                               # however, there is no substantive explanation as to how it was calculated/obtained.
               max_dist = 304.8, # m, expected stopping distance for a bust from 60 mph calculated from google map offramp length measurements of I-5.
               a_prof_path =a_prof_path, # based on the Braunschweig drive cycle @NREL_Drive_Cycles
               max_acc = .4, # m/s^2, chosen due to it being a decent extension of existing drive cycle. subject to change.
               max_dt = .5, #s, timestep for that extension. 
               max_P = 300000 # W, @Siemens_Traction_Motor 
              )

# Saving the object.
XDE60.save(bus_directory + "XDE60.txt")


# The XDE60 @Siemens_Traction_Motor
# Cooling system: @E-Fan_Cooling
# Approx 10080 watts at max cooling
# HVAC system, @Thermoking_TE18
# Approx 3k watts at max hvac
# thus, a reasonable nominal load is 8k? bottom is 4k?
XDE60_ESS = op.ESS(motor_eff = .91, # @Gallet. No clear source on where these drivetrain, inverter, and motor efficiencies came from. Can't find TB200 specs yet.
                   inverter_eff = .97, # @Gallet
                   aux_eff = .89, # Assumption.
                   regen_eff = .6, # Assumption.
                   simple_load = 8000,
                   max_regen = -200000 #w, finally, some good sourcable values.@XDE35_Manual
                   # W @Siemens_Traction_Motor
                  )
XDE60_ESS.save(ESS_directory + "XDE60_ESS.txt")

'./KC_Example_Data/Saved_Objects/ESSes/XDE60_ESS.txt'

In [5]:
# XDE40/XT40 BEV version: 
XDE40 =op.Bus(bus_mass = 13835, #kg, @NF_Xcelsior
               frontal_width = 2.6, #m,@NF_Xcelsior
               frontal_height = 3.2, #m, @NF_Xcelsior
               drag_coeff = .79, # Unitless. Many suspect papers cite .6, but best bet was one citing ORNL bus database saying .79, @Gao_Et_Al
               friction_coeff = .01, # unitless, otherwise known as rolling resistance. @Rolling_Resistance
               braking_accel = 1.5, #1.5m/s^2, handbrake required to stop from 20 mph, over no set distance. emergency brake must be capable of up to 6.5m/s^2 (20 mph over 20 ft).@APTA_Braking_Standards
               br_factor = .5, # driver braking aggression. Variable based on driver.
               i_factor = 1.1, # unitless, accounts for wheels, driveshaft, etc, cited from @Gallet
                               # however, there is no substantive explanation as to how it was calculated/obtained.
               max_dist = 304.8, # m, expected stopping distance for a bust from 60 mph calculated from google map offramp length measurements of I-5.
               a_prof_path =a_prof_path, # based on the Braunschweig drive cycle @NREL_Drive_Cycles
               max_acc = .4, # m/s^2, chosen due to it being a decent extension of existing drive cycle. subject to change.
               max_dt = .5, #s, timestep for that extension. 
               max_P = 300000 # W, @Siemens_Traction_Motor 
              )

# Saving the object.
XDE40.save(bus_directory + "XT40.txt")


# The XDE60 @Siemens_Traction_Motor
# Cooling system: @E-Fan_Cooling
# Approx 10080 watts at max cooling
# HVAC system, @Thermoking_TE18
# Approx 3k watts at max hvac
# thus, a reasonable nominal load is 8k? bottom is 4k?
XDE40_ESS =  op.ESS(motor_eff = .91, # @Gallet. No clear source on where these drivetrain, inverter, and motor efficiencies came from. Can't find TB200 specs yet.
                   inverter_eff = .97, # @Gallet
                   aux_eff = .89, # Assumption.
                   regen_eff = .6, # Assumption.
                   simple_load = 8000,
                   max_regen = -200000 #w, finally, some good sourcable values.@XDE35_Manual
                   # W @Siemens_Traction_Motor
                  )

XDE40_ESS.save(ESS_directory + "XT40_ESS.txt")

'./KC_Example_Data/Saved_Objects/ESSes/XT40_ESS.txt'